In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [47]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [48]:
df = fetch_hourly_rides(12)

2025-03-04 12:25:59,802 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 12:25:59,802 INFO: Initializing external client
2025-03-04 12:25:59,802 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:26:00,484 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.51s) 


In [49]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-04 03:00:00+00:00,46,0
1,2025-03-04 01:00:00+00:00,99,0
2,2025-03-04 01:00:00+00:00,89,0
3,2025-03-04 03:00:00+00:00,164,7
4,2025-03-04 03:00:00+00:00,28,0
...,...,...,...
4513,2025-03-04 00:00:00+00:00,214,0
4514,2025-03-04 02:00:00+00:00,244,0
4515,2025-03-04 03:00:00+00:00,241,0
4516,2025-03-04 03:00:00+00:00,107,4


In [50]:
df_pred = fetch_predictions(12)

2025-03-04 12:26:08,325 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 12:26:08,340 INFO: Initializing external client
2025-03-04 12:26:08,340 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:26:08,826 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.47s) 


In [51]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,252,0.0,2025-03-04 16:00:00+00:00
1,243,1.0,2025-03-04 16:00:00+00:00
2,224,3.0,2025-03-04 16:00:00+00:00
3,117,0.0,2025-03-04 16:00:00+00:00
4,69,0.0,2025-03-04 16:00:00+00:00
...,...,...,...
748,142,158.0,2025-03-04 18:00:00+00:00
749,41,3.0,2025-03-04 18:00:00+00:00
750,226,2.0,2025-03-04 18:00:00+00:00
751,190,0.0,2025-03-04 18:00:00+00:00


In [52]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [53]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-04 16:00:00+00:00,83,0,0.0
1,2025-03-04 16:00:00+00:00,36,0,1.0
2,2025-03-04 16:00:00+00:00,177,0,0.0
3,2025-03-04 16:00:00+00:00,236,448,221.0
4,2025-03-04 16:00:00+00:00,127,0,1.0
...,...,...,...,...
497,2025-03-04 17:00:00+00:00,199,0,0.0
498,2025-03-04 17:00:00+00:00,117,0,0.0
499,2025-03-04 17:00:00+00:00,98,0,0.0
500,2025-03-04 17:00:00+00:00,75,57,58.0


In [54]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [55]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
47,2025-03-04 16:00:00+00:00,2,0,0.0,0.0
358,2025-03-04 17:00:00+00:00,2,0,0.0,0.0
121,2025-03-04 16:00:00+00:00,3,1,0.0,-1.0
275,2025-03-04 17:00:00+00:00,3,0,0.0,0.0
142,2025-03-04 16:00:00+00:00,4,2,20.0,18.0
...,...,...,...,...,...
444,2025-03-04 17:00:00+00:00,261,58,28.0,-30.0
214,2025-03-04 16:00:00+00:00,262,111,35.0,-76.0
270,2025-03-04 17:00:00+00:00,262,101,93.0,-8.0
164,2025-03-04 16:00:00+00:00,263,113,35.0,-78.0


In [56]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-04 16:00:00+00:00,83,0,0.0,0.0
1,2025-03-04 16:00:00+00:00,36,0,1.0,1.0
2,2025-03-04 16:00:00+00:00,177,0,0.0,0.0
3,2025-03-04 16:00:00+00:00,236,448,221.0,-227.0
4,2025-03-04 16:00:00+00:00,127,0,1.0,1.0
...,...,...,...,...,...
497,2025-03-04 17:00:00+00:00,199,0,0.0,0.0
498,2025-03-04 17:00:00+00:00,117,0,0.0,0.0
499,2025-03-04 17:00:00+00:00,98,0,0.0,0.0
500,2025-03-04 17:00:00+00:00,75,57,58.0,1.0


In [57]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [58]:
mae_by_hour["MAE"].mean()

14.165338645418327